In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols, glm
import numpy as np
from IPython import display

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Data Import
raw_data = pd.read_csv('/content/drive/MyDrive/student_math_grades.csv')
raw_data.head(2)

,school,sex,age,address,famsize,pstatus,medu,fedu,mjob,fjob,...,higher,internet,famrel,freetime,goout,dalc,walc,health,absences,grades
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,4,3,4,1,1,3,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,5,3,3,1,1,3,4,6


In [ ]:
# Sanity Check
print("Shape: " + str(raw_data.shape))
SC = []
for i in raw_data.columns:
  SC.append(raw_data[i].value_counts())
print(SC)

Shape: (395, 30)
[GP    349
MS     46
Name: school, dtype: int64, F    208
M    187
Name: sex, dtype: int64, 16    104
17     98
18     82
15     82
19     24
20      3
22      1
21      1
Name: age, dtype: int64, U    307
R     88
Name: address, dtype: int64, GT3    281
LE3    114
Name: famsize, dtype: int64, T    354
A     41
Name: pstatus, dtype: int64, 4    131
2    103
3     99
1     59
0      3
Name: medu, dtype: int64, 2    115
3    100
4     96
1     82
0      2
Name: fedu, dtype: int64, other       141
services    103
at_home      59
teacher      58
health       34
Name: mjob, dtype: int64, other       217
services    111
teacher      29
at_home      20
health       18
Name: fjob, dtype: int64, course        145
home          109
reputation    105
other          36
Name: reason, dtype: int64, mother    273
father     90
other      32
Name: guardian, dtype: int64, 1    257
2    107
3     23
4      8
Name: traveltime, dtype: int64, 2    198
1    105
3     65
4     27
Name: study

In [ ]:
# Data Cleaning
raw_data['school'].replace({"GP":1, "MS":0}, inplace = True)
raw_data['sex'].replace({"M":1, "F":0}, inplace = True)
raw_data['address'].replace({"U":1, "R":0}, inplace = True)
raw_data['famsize'].replace({"LE3":1, "GT3":0}, inplace = True)
raw_data['pstatus'].replace({"T":1, "A":0}, inplace = True)
raw_data['schoolsup'].replace({"yes":1, "no":0}, inplace = True)
raw_data['famsup'].replace({"yes":1, "no":0}, inplace = True)
raw_data['paid'].replace({"yes":1, "no":0}, inplace = True)
raw_data['activities'].replace({"yes":1, "no":0}, inplace = True)
raw_data['nursery'].replace({"yes":1, "no":0}, inplace = True)
raw_data['higher'].replace({"yes":1, "no":0}, inplace = True)
raw_data['internet'].replace({"yes":1, "no":0}, inplace = True)
df = pd.concat([raw_data, pd.get_dummies(raw_data['mjob'], prefix = 'm', drop_first=True)], axis = 1)
df = pd.concat([df, pd.get_dummies(raw_data['fjob'], prefix = 'f', drop_first=True)], axis = 1)
df = pd.concat([df, pd.get_dummies(raw_data['reason'], prefix = 'reason', drop_first=True)], axis = 1)
df = pd.concat([df, pd.get_dummies(raw_data['guardian'], prefix = 'guardian', drop_first=True)], axis = 1)
df.drop(columns = ['mjob', 'fjob', 'reason', 'guardian'], inplace = True)

***Linear Reg - Backward Stepwise Regression***

In [ ]:
# Linear Reg - Backward Stepwise Regression.
mod0 = ols('grades ~ medu + failures', data = df).fit()
print(mod0.summary())

                            OLS Regression Results                            
Dep. Variable:                 grades   R-squared:                       0.148
Model:                            OLS   Adj. R-squared:                  0.144
Method:                 Least Squares   F-statistic:                     34.13
Date:                Wed, 04 May 2022   Prob (F-statistic):           2.16e-14
Time:                        07:26:12   Log-Likelihood:                -1129.5
No. Observations:                 395   AIC:                             2265.
Df Residuals:                     392   BIC:                             2277.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      9.4821      0.621     15.267      0.0

Adj. R-squared: 0.144 is low in terms of Marketing, but ***acceptable*** in Sociology.

**Falk and Miller (1992)** recommended that R2 values should be equal to or greater than 0.10 for the variance explained of a particular endogenous construct to be deemed adequate.
**Cohen (1988)** suggested R2 values for endogenous latent variables are assessed as follows: 0.26 (substantial), 0.13 (moderate), 0.02 (weak). 

*The there’s only one variable having a negative impact on the outcome, “failures”, which means number of past class failures. It indicates that the number of past class failures causes final failure.*

*Based on the model above, the there’s two variable having significant impact on the outcome, “medu” and “failures”.*

*According to the attribute information, we know the mother's education is out of control. Therefore, the only way is decreasing the number of past class failures of students.*

**OR% in Logistic Regressiona**

In [ ]:
df['grades'].replace({20:1, 19:1, 18:1, 17:1, 16:1, 15:1, 14:1, 13:1, 
                              12:1, 11:1, 10:1, 9:1, 8:1, 7:1, 6:0, 5:1, 4:1,
                              3:1, 2:1, 1:0}, inplace = True)
df['grades'].head()

0    0
1    0
2    1
3    1
4    1
Name: grades, dtype: int64

In [ ]:
mod1 = glm('grades ~ school + sex + age + address + famsize + pstatus + medu + fedu + \
traveltime + studytime + failures + schoolsup + famsup + paid + activities + \
nursery + higher + internet + famrel + freetime + goout + dalc + walc + health \
+ absences + m_health + m_other + m_services + m_teacher + f_health + f_other + \
f_services + f_teacher + reason_home + reason_other + reason_reputation + \
guardian_mother + guardian_other', data = df).fit()
print(mod1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 grades   No. Observations:                  395
Model:                            GLM   Df Residuals:                      356
Model Family:                Gaussian   Df Model:                           38
Link Function:               identity   Scale:                         0.10635
Method:                          IRLS   Log-Likelihood:                -97.352
Date:                Wed, 04 May 2022   Deviance:                       37.861
Time:                        07:26:12   Pearson chi2:                     37.9
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.9078      0.37

In [ ]:
# OR%
(np.exp(mod1.params)-1) * 100

Intercept            147.882754
school                -9.190385
sex                   10.399736
age                   -2.012325
address                1.288653
famsize                1.292575
pstatus                1.908479
medu                   0.012812
fedu                   0.561831
traveltime            -2.681922
studytime              3.382105
failures              -6.068424
schoolsup              2.336930
famsup                -6.566244
paid                   8.702356
activities            -1.581724
nursery               -0.364149
higher                11.365173
internet               1.370934
famrel                 1.402131
freetime               2.472229
goout                 -3.714487
dalc                  -1.573432
walc                   5.141430
health                -0.880035
absences               0.869592
m_health               9.579293
m_other                3.730081
m_services             6.261773
m_teacher              2.085347
f_health              12.892039
f_other 